In [2]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
import string
from sklearn.model_selection import train_test_split  
from random import shuffle
import sklearn.metrics
import os
import scipy
from sklearn import svm
from sklearn.externals import joblib
from sklearn.pipeline import Pipeline
import pickle
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier 
from sklearn.linear_model import SGDClassifier

In [1]:
train=pd.read_csv("train123.csv",header=None,encoding='ISO-8859-1')
test=pd.read_csv("uniondata.csv")
train.columns=["label","1","2","3","4","tweet"]
x=train["tweet"].tolist()
y=train["label"].tolist()
X_train, X_test, y_train, y_test = train_test_split(x,y,test_size=0.20, random_state=42)
test=test["tweets"].tolist()
print(test.head())
print(train.head())

NameError: name 'pd' is not defined

In [12]:
def preprocessTweets(tweet):
    #Convert www.* or https?://* to URL
    tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','URL',tweet)
    #Convert @username to __HANDLE
    tweet = re.sub('@[^\s]+','__HANDLE',tweet)  
    #Replace #word with word
    tweet = re.sub(r'#([^\s]+)', r'\1', tweet)
    #trim
    tweet = tweet.strip('\'"')
    # Repeating words like akashhhhhhhhhhh
    rpt_regex = re.compile(r"(.)\1{1,}", re.IGNORECASE)
    tweet = rpt_regex.sub(r"\1\1", tweet)
    #Emoticons
    emoticons = \
    [
     ('__positive__',[ ':-)', ':)', '(:', '(-:', \
                       ':-D', ':D', 'X-D', 'XD', 'xD', \
                       '<3', ':\*', ';-)', ';)', ';-D', ';D', '(;', '(-;', ] ),\
     ('__negative__', [':-(', ':(', '(:', '(-:', ':,(',\
                       ':\'(', ':"(', ':((', ] ),\
    ]
    def replace_parenth(arr):
       return [text.replace(')', '[)}\]]').replace('(', '[({\[]') for text in arr]
    
    def regex_join(arr):
        return '(' + '|'.join( arr ) + ')'

    emoticons_regex = [ (repl, re.compile(regex_join(replace_parenth(regx))) ) \
            for (repl, regx) in emoticons ]
    
    for (repl, regx) in emoticons_regex :
        tweet = re.sub(regx, ' '+repl+' ', tweet)
     #Convert to lower case
    tweet = tweet.lower()
    return tweet
#Stemming of Tweets
def stem(tweet):
        stemmer = nltk.stem.PorterStemmer()
        tweet_stem = ''
        words = [word if(word[0:2]=='__') else word.lower() \
                    for word in tweet.split() \
                    if len(word) >= 3]
        words = [stemmer.stem(w) for w in words] 
        tweet_stem = ' '.join(words)
        return tweet_stem

In [13]:
def processTweets(X_train, X_test):
        X_train = [stem(preprocessTweets(tweet)) for tweet in X_train]
        X_test = [stem(preprocessTweets(tweet)) for tweet in X_test]
        return X_train,X_test

In [14]:
X_train, X_test = processTweets(X_train, X_test)

In [15]:
def classifier(X_train,y_train):
        vec = TfidfVectorizer(min_df=5, max_df=0.95, sublinear_tf = True,use_idf = True,ngram_range=(1, 2))
        svm_clf =svm.LinearSVC(C=0.1)
        vec_clf = Pipeline([('vectorizer', vec), ('pac', svm_clf)])
        vec_clf.fit(X_train,y_train)
        joblib.dump(vec_clf, 'svmClassifier.pkl', compress=3)
        return vec_clf
vec_clf1 = classifier(X_train,y_train)

C:\Users\kools\Anaconda3\lib\site-packages\sklearn\svm\base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [16]:
y_pred = vec_clf1.predict(X_test)
print(sklearn.metrics.classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.83      0.80      0.81    159494
           4       0.81      0.83      0.82    160506

   micro avg       0.82      0.82      0.82    320000
   macro avg       0.82      0.82      0.82    320000
weighted avg       0.82      0.82      0.82    320000



In [17]:
def classifiersgd(X_train,y_train):
        vec = TfidfVectorizer(min_df=5, max_df=0.95, sublinear_tf = True,use_idf = True,ngram_range=(1, 2))
        svm_clf =SGDClassifier(alpha=0.0001,max_iter=1000)
        vec_clf = Pipeline([('vectorizer', vec), ('pac', svm_clf)])
        vec_clf.fit(X_train,y_train)
        joblib.dump(vec_clf, 'sgdClassifier.pkl', compress=3)
        return vec_clf
vec_clf2 = classifiersgd(X_train,y_train)

C:\Users\kools\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Specify tol to silence this warning.
  FutureWarning)


In [18]:
y_pred = vec_clf2.predict(X_test)
print(sklearn.metrics.classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.79      0.77      0.78    159494
           4       0.78      0.79      0.78    160506

   micro avg       0.78      0.78      0.78    320000
   macro avg       0.78      0.78      0.78    320000
weighted avg       0.78      0.78      0.78    320000



In [25]:
def classifierRFC(X_train,y_train):
        vec = TfidfVectorizer(min_df=5, max_df=0.95, sublinear_tf = True,use_idf = True,ngram_range=(1, 2))
        svm_clf =RandomForestClassifier(n_estimators=100,max_depth=5)
        vec_clf = Pipeline([('vectorizer', vec), ('pac', svm_clf)])
        vec_clf.fit(X_train,y_train)
        joblib.dump(vec_clf, 'RFCClassifier.pkl', compress=3)
        return vec_clf
vec_clf3 = classifierRFC(X_train,y_train)

In [26]:
y_pred = vec_clf3.predict(X_test)
print(sklearn.metrics.classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.67      0.75      0.71    159494
           4       0.72      0.64      0.68    160506

   micro avg       0.70      0.70      0.70    320000
   macro avg       0.70      0.70      0.69    320000
weighted avg       0.70      0.70      0.69    320000



In [21]:
def classifierNB(X_train,y_train):
        vec = TfidfVectorizer(min_df=5, max_df=0.95, sublinear_tf = True,use_idf = True,ngram_range=(1, 2))
        svm_clf =MultinomialNB()
        vec_clf = Pipeline([('vectorizer', vec), ('pac', svm_clf)])
        vec_clf.fit(X_train,y_train)
        joblib.dump(vec_clf, 'MNBClassifier.pkl', compress=3)
        return vec_clf
vec_clf4 = classifierNB(X_train,y_train)

In [22]:
y_pred = vec_clf4.predict(X_test)
print(sklearn.metrics.classification_report(y_test, y_pred)) 

              precision    recall  f1-score   support

           0       0.80      0.80      0.80    159494
           4       0.80      0.79      0.80    160506

   micro avg       0.80      0.80      0.80    320000
   macro avg       0.80      0.80      0.80    320000
weighted avg       0.80      0.80      0.80    320000



In [32]:
def classifierKNC(X_train,y_train):
        vec = TfidfVectorizer(min_df=5, max_df=0.95, sublinear_tf = True,use_idf = True,ngram_range=(1, 2))
        svm_clf =KNeighborsClassifier(n_neighbors=5)
        vec_clf = Pipeline([('vectorizer', vec), ('pac', svm_clf)])
        vec_clf.fit(X_train,y_train)
        joblib.dump(vec_clf, 'KNCClassifier.pkl', compress=3)
        return vec_clf
vec_clf6 = classifierKNC(X_train,y_train)

In [ ]:
y_pred = vec_clf6.predict(X_test)
print(sklearn.metrics.classification_report(y_test, y_pred))

In [ ]:
test=pd.read_csv("test123.csv",header=None,encoding='ISO-8859-1')
test.columns=["label","1","2","3","4","tweet"]
test_x=test["tweet"][test.label!=2].tolist()
test_y=test["label"][test.label!=2].tolist()

In [ ]:
sklearn.metrics.accuracy_score(test_y,y_pred)

In [ ]:
classifier_f = open("svmClassifier.pkl", "rb")
classifier = joblib.load(classifier_f)
classifier_f.close()